In [1]:
import pandas as pd

def xymaker(df):
    df["long"] = df["long"].round(5)
    df["lat"] = df["lat"].round(5)
    
    unique_df = df.drop_duplicates(subset=["long", "lat"])
    return unique_df[["long", "lat"]].values.tolist()

In [2]:
import pandas as pd

def xymaker(df):
    df["long"] = df["long"].round(5)
    df["lat"] = df["lat"].round(5)
    
    unique_df = df.drop_duplicates(subset=["long", "lat"])
    return unique_df[["long", "lat"]].values.tolist()

In [3]:
home_data = pd.read_csv("main.csv")

home_list = xymaker(home_data)

home_list[1:10]

/tmp/ipykernel_506346/886316247.py:1: DtypeWarning: Columns (10,13,14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  home_data = pd.read_csv("main.csv")


[[127.05609, 37.48463],
 [127.07613, 37.49629],
 [127.05785, 37.48029],
 [127.06903, 37.48791],
 [127.07087, 37.48914],
 [127.03827, 37.51509],
 [127.02898, 37.51386],
 [127.03842, 37.50953],
 [127.05885, 37.49664]]

In [4]:
apartment_list = [
    [127.0206, 37.5341],  # 강남구 - 압구정 현대14차
    [127.0450, 37.4935],  # 강남구 - 타워팰리스1차
    [126.9959, 37.5133],  # 서초구 - 아크로리버파크
    [127.0155, 37.4948],  # 서초구 - 래미안 원베일리
    [127.0950, 37.5101],  # 송파구 - 잠실 리센츠
    [127.0812, 37.5111],  # 송파구 - 잠실 엘스
    [127.0004, 37.5350],  # 용산구 - 한남더힐
    [127.0050, 37.5345],  # 용산구 - 나인원한남
    [127.0445, 37.5446],  # 성동구 - 아크로서울포레스트
    [127.0430, 37.5440],  # 성동구 - 갤러리아 포레
    [126.9450, 37.5412],  # 마포구
    [126.9683, 37.5705],  # 종로구
    [126.9950, 37.5636],  # 중구
    [126.9810, 37.5030],  # 동작구
    [126.8700, 37.5300],  # 양천구
    [126.8270, 37.5610],  # 강서구
    [127.0670, 37.6440],  # 노원구
    [127.0260, 37.6260],  # 강북구
    [127.0440, 37.6670],  # 도봉구
    [126.9180, 37.5990],  # 은평구
    [126.9090, 37.5790],  # 서대문구
    [127.0870, 37.5880],  # 중랑구
    [127.0650, 37.5940],  # 동대문구
    [127.0230, 37.6030],  # 성북구
    [127.1030, 37.5460],  # 광진구
    [126.8890, 37.4950],  # 구로구
    [126.9000, 37.4560],  # 금천구
    [126.9510, 37.4840],  # 관악구
    [127.1540, 37.5610],  # 강동구
]
 
    

In [5]:
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = R * c
    return distance

In [6]:
from collections import defaultdict

def count_apartments_within_range(apt_coords, reference_coords, distance_threshold):
   
    count_dict = defaultdict(int)

    for apt_lat, apt_lon in apt_coords:
        for ref_lat, ref_lon in reference_coords:
            distance = haversine(apt_lat, apt_lon, ref_lat, ref_lon)
            if distance <= distance_threshold:
                count_dict[(apt_lat, apt_lon)] += 1  

    return count_dict


In [7]:
apt_coords = home_list 
reference_coords = apartment_list  


count_dict = count_apartments_within_range(apt_coords, reference_coords, 4)


home_data['아파트군'] = home_data.apply(lambda row: count_dict.get((row['long'], row['lat']), 0), axis=1)

In [8]:
home_data['아파트군'].value_counts()

2     355343
3     316901
1     120209
4     106557
5      73871
6      48740
7      40969
8      27710
0      24444
9      11956
10      1498
Name: 아파트군, dtype: int64

In [9]:
home_data.groupby("아파트군")["target"].mean()

아파트군
0      55178.355506
1      51734.071243
2      49657.588561
3      52319.140072
4      57906.200325
5      72805.738977
6      86930.519922
7      98164.810784
8      95286.980512
9      88900.829458
10    108745.728304
Name: target, dtype: float64

In [10]:
correlation = home_data["아파트군"].corr(home_data["target"])
print(f"상관계수: {correlation}")


상관계수: 0.26867024429380487
